In [1]:
cd /app

/app


/home/bar3126/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
!pwd

/app


In [4]:
from bullyguard.data_processing.dataset_cleaners import SpellCorrectionDatasetCleaner
from bullyguard.utils.utils import SpellCorrectionModel

In [5]:
spell_correction_model = SpellCorrectionModel()
cleaner = SpellCorrectionDatasetCleaner(spell_correction_model)

In [6]:
text = "whereis th elove hehad dated forImuch of thepast who couqdn'tread in sixthgrade and ins pired him." 

In [7]:
cleaner(text)

"where is the love he had dated for much of the past who couldn't read in sixth grade and inspired him"

In [3]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

def reduce_dataset_stratified(file_path, output_path, target_columns=None, separator=','):
    # Read the file
    df = pd.read_csv(file_path, sep=separator)
    
    if target_columns:
        # If target columns are specified, use StratifiedKFold
        if isinstance(target_columns, str):
            target_columns = [target_columns]
        
        # Create a combined target for stratification
        df['combined_target'] = df[target_columns].apply(lambda row: '_'.join(row.astype(str)), axis=1)
        
        skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
        
        # Get the indices for the stratified split
        for _, reduced_indices in skf.split(df, df['combined_target']):
            df_reduced = df.iloc[reduced_indices]
            break
        
        # Remove the combined target column
        df_reduced = df_reduced.drop('combined_target', axis=1)
    else:
        # If no target columns are specified, use random sampling
        df_reduced = df.sample(frac=0.5, random_state=42)
    
    # Save the reduced dataset
    df_reduced.to_csv(output_path, sep=separator, index=False)
    
    print(f"Original dataset size: {len(df)}")
    print(f"Reduced dataset size: {len(df_reduced)}")
    print(f"Reduced dataset saved to: {output_path}")


In [ ]:
# For the .tsv file (assuming 'hd', 'cv', 'vo' are target variables)
reduce_dataset_stratified('data/raw/ghc/ghc_test.tsv', 'ghc_test.tsv1', 
                          target_columns=['hd', 'cv', 'vo'], separator='\t')

reduce_dataset_stratified('data/raw/ghc/ghc_train.tsv', 'ghc_train.tsv1', 
                          target_columns=['hd', 'cv', 'vo'], separator='\t')

reduce_dataset_stratified('data/raw/ghc/GabHateCorpus_annotations.tsv', 'GabHateCorpus_annotations.tsv1', 
                          target_columns=['hd', 'cv', 'vo'], separator='\t')

reduce_dataset_stratified('data/raw/jigsaw-toxic-comment/train.csv', 'train1.csv', 
                          target_columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

reduce_dataset_stratified('data/raw/jigsaw-toxic-comment/test_labels.csv', 'test_labels1.csv', 
                          target_columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

# For the second .csv file (no clear target variable, using random sampling)
reduce_dataset_stratified('data/raw/jigsaw-toxic-comment/test.csv', 'test1.csv')

# For the third .csv file (with multiple target columns)
reduce_dataset_stratified('path_to_your_third_csv_file.csv', 'reduced_csv_file3.csv', 
                          target_columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

# For the last .csv file (tweet dataset, using 'cyberbullying_type' as target)
reduce_dataset_stratified('data/raw/twitter/cyberbullying_tweets.csv', 'cyberbullying_tweets1.csv', 
                          target_columns='cyberbullying_type')